In [60]:
import pandas as pd
import pymc as pm
import numpy as np
from scipy.special import logsumexp

%reload_ext watermark

In [9]:
%watermark --iversions

pandas: 1.4.2
pymc  : 4.0.0b6



In [10]:
path = "../../../data/users/summaries/combined/"

## Setup for a Bayesian model

Let's try to see if there is any relationship between probability of "success" that a user will choose to post an entry of a specific topic as a function of the user attributes.

This will be the setup:

There are $n$ users that have $m$ covariates and post across $k$ topics. Then:

$Y$ (n x k) where $Y_i^j$ denotes observed count of topic $i$ posted by user $j$

$c$ (n x 1) where $c^j$ denotes total number of posts made by user $j$

$p$ (n x k) where $p_i^j$ denotes the probability that a post made by user $j$ is of topic $i$

$X$ (n x m) where $X_m^j$ denotes attribute $m$ of user $j$

$Y \sim  Multinomial(p, c)$

$p = softmax(Z)$

$Z = \alpha + X \beta$

$\beta$ (k x m) $\sim Normal(0, scale=0.001)$

$\alpha$ (k x 1) $\sim Normal(0, scale=0.001)$

 



In [100]:
data = pd.read_csv(path + "for_multinomial.csv")

In [101]:
subset = data[data['total'] > 10]

feature_cols = ['no_posts', "no_comments", "post_karma", "comment_karma", "avg_post_karma", "avg_comment_karma", "activity_window", "longevity", "indirect_pg_rank"]
topic_cols = [str(i) for i in range(25)]
total_col = ['total']

In [123]:
features = subset[feature_cols].values
counts = subset[total_col].values

n_users, m_features = features.shape
k_topics = len(topic_cols)

In [124]:
t = subset[topic_cols].values / subset[topic_cols].values.sum(axis=1).reshape(-1, 1)

In [125]:
topics = t.reshape(1, n_users, k_topics)

In [79]:
a = np.ones((n_users, 1))
b = np.ones((m_features, k_topics))
Z = features @ b + a
sums = logsumexp(Z, axis=1, keepdims = True)
p = np.exp(Z - sums)
p

array([[0.04, 0.04, 0.04, ..., 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, ..., 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, ..., 0.04, 0.04, 0.04],
       ...,
       [0.04, 0.04, 0.04, ..., 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, ..., 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, ..., 0.04, 0.04, 0.04]])

In [177]:
features = np.array([[1, 0, 1], [1, 1, 1], [0, 0, 1], [0, 1, 0]])
counts = np.array([5, 2, 4, 1])

beta = np.array([[1, 3], [3, 7], [8, 3]])

Z = features @ beta

sums = logsumexp(Z, axis=1, keepdims = True)
p = np.exp(Z - sums)


topics = np.round((p.T * counts).T,0)


n_users = 4
m_features = 3
k_topics = 2



In [182]:
with pm.Model() as model:
    data = pm.ConstantData("features", features)
    totals  = pm.ConstantData("counts", counts)
    Y = pm.ConstantData("topics", topics)
            
    β = pm.Normal("coefficients", 0, tau=1e-6, shape=(m_features, k_topics))
    
    Z = pm.math.dot(data, β)    
    sums = pm.math.logsumexp(Z, axis=1, keepdims = True)
    p = pm.math.exp(Z - sums)
            
    pm.Multinomial("outcomes", p = p, n = totals, observed = Y)
        
    trace = pm.sample(1000, chains=1)
    

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [coefficients]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 39 seconds.


In [181]:
import arviz as az

az.summary(trace)

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
b,-177.013,799.882,-1804.242,978.256,529.961,437.109,2.0,27.0,NaN
"intercept[0, 0]",2.610,11.239,-15.704,22.645,7.833,6.596,2.0,26.0,NaN
"intercept[1, 0]",-1.213,10.216,-16.185,19.011,1.624,1.157,40.0,87.0,NaN
"intercept[2, 0]",1.352,9.329,-17.124,17.144,1.175,0.834,62.0,116.0,NaN
"intercept[3, 0]",-0.894,10.459,-20.653,15.699,1.366,0.970,58.0,85.0,NaN
"coefficients[0, 0]",-297.344,989.502,-2118.199,1513.532,688.755,579.606,2.0,24.0,NaN
"coefficients[0, 1]",-431.485,940.377,-1777.961,1595.929,487.001,375.593,4.0,28.0,NaN
"coefficients[1, 0]",-712.160,1055.380,-2728.690,916.403,506.678,385.570,5.0,65.0,NaN
"coefficients[1, 1]",352.010,1186.478,-1968.606,2416.579,529.856,398.916,5.0,21.0,NaN
"coefficients[2, 0]",339.841,820.078,-900.306,2081.368,361.557,271.741,6.0,45.0,NaN
